<a href="https://colab.research.google.com/github/AliciaAPerez/Death_Analytics_using_CDC_Data_with_Machine_Learning/blob/alicia/Death_Machine_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-04-20 20:23:11--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.4’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2021-04-20 20:23:12 (6.02 MB/s) - ‘postgresql-42.2.9.jar.4’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#function for other data sets to be cleaned up - only can be used for 2014 and 2015 and newer, older pages use a different education marker
def data_clean_up(data):
  data = data.drop("education_1989_revision","education_reporting_flag","age_substitution_flag","age_recode_52","age_recode_27","age_recode_12", "infant_age_recode_22",
"place_of_death_and_decedents_status","method_of_disposition","autopsy", "activity_code", "place_of_injury_for_causes_w00_y34_except_y06_and_y07_", "358_cause_recode", "113_cause_recode",
"130_infant_cause_recode", "39_cause_recode", "bridged_race_flag", "race_imputation_flag", "race_recode_3", "race_recode_5", "hispanic_origin");
  data = data.filter((data.detail_age_type == 1))
  data = data.drop("detail_age_type")
  data = data.drop("number_of_entity_axis_conditions","entity_condition_4","entity_condition_5","entity_condition_6","entity_condition_7","entity_condition_8",
                                            "entity_condition_9","entity_condition_10","entity_condition_11","entity_condition_12","entity_condition_13","entity_condition_14",
                                            "entity_condition_15","entity_condition_16","entity_condition_17","entity_condition_18","entity_condition_19","entity_condition_20",
                                            "number_of_record_axis_conditions", "record_condition_1", "record_condition_2","record_condition_3","record_condition_4","record_condition_5",
                                            "record_condition_6","record_condition_7","record_condition_8","record_condition_9","record_condition_10","record_condition_11","record_condition_12",
                                            "record_condition_13","record_condition_14","record_condition_15","record_condition_16","record_condition_17","record_condition_18","record_condition_19",
                                            "record_condition_20",)
  data = data.withColumnRenamed("resident_status","resident").withColumnRenamed("education_2003_revision","education").withColumnRenamed("detail_age", "age").withColumnRenamed("current_date_year", "year").withColumnRenamed("icd_code_10th_revision", "icd_death_code").withColumnRenamed("hispanic_originrace_recode", "hispanic")
  data = data.withColumn("resident",col("resident").cast("integer")).withColumn("education",col("education").cast("integer")).withColumn("month_of_death",col("month_of_death").cast("integer")).withColumn("age",col("age").cast("integer")).withColumn("day_of_week_of_death",col("day_of_week_of_death").cast("integer")).withColumn("current_data_year",col("current_data_year").cast("integer")).withColumn("manner_of_death",col("manner_of_death").cast("integer")).withColumn("race",col("race").cast("integer")).withColumn("hispanic",col("hispanic").cast("integer"))
  data = data.fillna(value="0",subset=["entity_condition_1"]).fillna(value="0",subset=["entity_condition_2"]).fillna(value="0",subset=["entity_condition_3"])
  data = data.na.replace(["Y", "N", "U"], ["1","2","3"], "injury_at_work")
  data = data.na.replace(["M", "F"], ["1","2"], "sex")
  data = data.na.replace(["S", "M", "W", "D", "U"], ["1","2", "3", "4", "5"], "marital_status")
  data = data.select("icd_death_code","resident","education","month_of_death","sex", "age","marital_status","day_of_week_of_death","current_data_year","injury_at_work","manner_of_death","race","hispanic","entity_condition_1","entity_condition_2","entity_condition_3")
  data = data.na.drop()
  return data

In [ ]:
top_list = ["I251","C349","J449","G309","F03","J189","A419","E149","G20","X44","N185","K746","W19","X74","V892","X95"]

In [ ]:
# for a smaller data set, only grabbing the top 16 causes without renaming the rest of the top 60 causes to fit in these categories
# condering the top 16 has a lot of overlap in types
def code_clean_trim(data):
  data = data.filter(data.icd_death_code.isin(top_list))
  return data

In [ ]:
# UPLOAD TO DATABASE BEFORE RUNNING THIS FUNCTION
# Just personal preference that the database is slightly more clear with the code names as there are several to get renamed
def code_convert(data):
  data = data.na.replace(['I251', 'C349', 'J449', 'G309', 'F03', 'J189', 'A419', 'E149', 'G20', 'X44', 'N185', 'K746', 'W19', 'X74','V892', 'X95'],\
  ["1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16"], "icd_death_code")
  return data

In [ ]:
url ="https://aperezbucket.s3.us-east-2.amazonaws.com/2015_data.csv"
spark.sparkContext.addFile(url)
death_2015 = spark.read.csv(SparkFiles.get("2015_data.csv"), sep=",", header=True)

#run the function for clean_up and show that dataframe
death_2015_final = data_clean_up(death_2015)
# death_2015_final.show(5)

In [ ]:
  death_2015_clean1 = death_2015_final.na.replace(["C509","C259","C189","C61","C80"],\
                         ["C349","C349","C349","C349","C349",],"icd_death_code")

In [ ]:
  death_2015_clean2 = death_2015_final.na.replace(["I219","I500","I64","I250","I119","I48"],\
                         ["I251","I251","I251","I251","I251","I251"],"icd_death_code")

In [ ]:
# ONLY USE TO HAVE SMALLER DATA SET
# USE NEXT CELL FOR MORE DATA
death_2015_clean_trim = code_clean_trim(death_2015_clean2)
# death_2015_clean_trim.count()

In [ ]:
# TO GET ALL NUMBERS TO RUN PROPER MUST ANNOTATE EACH ONTO FIRST AND ADD AN ID COLUMN THEN LOAD IN SQL 
# IF USING MULTIPLE DATAFRAMES, COMBINE BEFORE DOING THIS
from pyspark.sql.functions import monotonically_increasing_id

# this is for when all the dataframes are combined
# death_numbered = death10.withColumn("id", monotonically_increasing_id() +1)

death_numbered = death_2015_clean_trim.withColumn("id", monotonically_increasing_id() +1)

# death_numbered = death_2015_clean.withColumn("id", monotonically_increasing_id() +1)

In [ ]:
# # CONNECT TO DATABASE AND LOAD IN DATA
# mode = "append"
# jdbc_url="jdbc:postgresql://database-1.ck7rneirj52d.us-east-2.rds.amazonaws.com:5432/Death_Database"
# config = {"user":"root", 
#           "password": "postgres", 
#           "driver":"org.postgresql.Driver"}

In [ ]:
# death_numbered.write.jdbc(url=jdbc_url, table='cdc_death_table', mode=mode, properties=config)

In [ ]:
# now we can replace all the data with numbers 
# death_final = code_convert(death_numbered)
# death_final.show(5)

In [ ]:
# convert to pandas
# death_machine = death_final.select("*").toPandas()
# death_machine

In [ ]:
# death_machine.value_counts("icd_death_code")

In [ ]:
ten_list = ["I251","C349","J449","G309","F03","J189","A419","E149","G20","X44"]

In [ ]:
top_ten = death_numbered.filter(death_numbered.icd_death_code.isin(ten_list))

In [ ]:
# now we can replace all the data with numbers 
death_final = code_convert(top_ten)
# death_final.show(5)

In [ ]:
# convert to pandas
death_machine = death_final.select("*").toPandas()
# death_machine

In [ ]:
death_machine2 = top_ten.select("*").toPandas()

In [ ]:
death_machine2.value_counts("icd_death_code")

icd_death_code
I251    428754
C349    126123
J449     94129
F03      92868
G309     84389
J189     36206
A419     32554
E149     25512
G20      22682
X44      19194
dtype: int64

In [ ]:
###MACHINE LEARNING SET UP VARIABLES
import sklearn
import sklearn.datasets
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
wanted_data = death_machine[["education","sex", "age", "marital_status","race", "hispanic"]]
# wanted_data

In [ ]:
X = wanted_data
y = death_machine["icd_death_code"]
print(X.shape, y.shape)

(962411, 6) (962411,)


In [ ]:
wanted_data2 = death_machine2[["education","sex", "age", "marital_status","race", "hispanic"]]
# wanted_data

In [ ]:
X2 = wanted_data2
y2 = death_machine2["icd_death_code"]
print(X2.shape, y2.shape)

(962411, 6) (962411,)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, random_state=42)

In [ ]:
X_train = X_train.astype(int)
X_test = X_test.astype(int)
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [ ]:
X2_train = X2_train.astype(int)
X2_test = X2_test.astype(int)
# y2_train = y2_train.astype(int)
# y2_test = y2_test.astype(int)

In [ ]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
X2_scaler = MinMaxScaler().fit(X_train)
X2_train_scaled = X2_scaler.transform(X_train)
X2_test_scaled = X2_scaler.transform(X_test)

In [ ]:
print(X_train_scaled.shape)
print(X_test_scaled.shape)
print(y_train.shape)
print(y_test.shape)

(721808, 6)
(240603, 6)
(721808,)
(240603,)


In [ ]:
print(X2_train_scaled.shape)
print(X2_test_scaled.shape)
print(y2_train.shape)
print(y2_test.shape)

(721808, 6)
(240603, 6)
(721808,)
(240603,)


In [ ]:
from tensorflow.keras.utils import to_categorical
label_encoder = LabelEncoder()
label_encoder.fit(y2_train)
encoded_y_train = label_encoder.transform(y2_train)
encoded_y_test = label_encoder.transform(y2_test)

In [ ]:
y2_train_categorical = to_categorical(encoded_y_train)
y2_test_categorical = to_categorical(encoded_y_test)

In [ ]:
y2_train_categorical.shape

(721808, 10)

In [ ]:
y_train.shape

(721808,)

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train_scaled, y_train)
clf.score(X_test_scaled, y_test)

0.4410252573741807

In [ ]:
import pickle
pickle.dump(clf, open('clf_death_model.pkl', 'wb'))
model1 = pickle.load(open('clf_death_model.pkl', 'rb'))
print(model1)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')


In [ ]:
a = X_test_scaled[0]
a

array([0.125     , 1.        , 0.08817635, 0.5       , 0.        ,
       0.        ])

In [ ]:
answer = model1.predict(X_test_scaled[0])

ValueError: ignored

In [ ]:
b = a[1, :, None]

In [ ]:
b= [1 0 0 0 0 0 0 0 0 0]

SyntaxError: ignored

In [ ]:
              [[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train_scaled, y_train_categorical)
rf.score(X_test_scaled, y_test_categorical)

0.1232985457371687

In [ ]:
import pickle
pickle.dump(rf, open('rf_death_model.pkl', 'wb'))
model2 = pickle.load(open('rf_death_model.pkl', 'rb'))
print(model2)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
train_scores = []
test_scores = []
for k in range(1, 6, 2):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_scaled, y_train_categorical)
    train_score = knn.score(X_train_scaled, y_train_categorical)
    test_score = knn.score(X_test_scaled, y_test_categorical)
    train_scores.append(train_score)
    test_scores.append(test_score)
    print(f"k: {k}, Train/Test Score: {train_score:.3f}/{test_score:.3f}")

In [ ]:
import pickle
pickle.dump(knn, open('knn_death_model.pkl', 'wb'))
model3 = pickle.load(open('knn_death_model.pkl', 'rb'))
print(model3)